In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui
from tqdm import tqdm

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains


In [2]:
# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s_time = time.time( )

from_day = 1 # 7일 전부터
to_day = 1 # 1일 전부터

a = datetime.now()-timedelta(days=to_day)
a = a.strftime('%Y-%m-%d')
b = datetime.now()-timedelta(days=from_day)
b = b.strftime('%Y-%m-%d')
print(b, a)

2021-05-24 2021-05-26


In [3]:
print("=" *80)
print("         민주당 권리당원게시판 크롤러입니다.")
print("=" *80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\daily\\{b}\\' # 운영체제 상관없이 바탕화면 'crawler-bot' 폴더에 작업

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

         민주당 권리당원게시판 크롤러입니다.
데이터 크롤링을 시작합니다.


In [4]:
# 저장될 파일 경로와 이름을 지정
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 새로 생성합니다.
try : 
    os.makedirs(f_dir)
except : pass
else : 
    os.chdir(f_dir)

fc_name=f_dir+'\\'+f'minju.csv'

In [5]:
# 크롤링 데이터가 들어갈 리스트 생성
domain2 = []    # 크롤링 도메인
keyword2 = []   # 크롤링 키워드
title2 = []     # 게시물 제목
user2 = []      # 게시물 작성자
url2 = []       # 게시물 url
date2 = []      # 게시물 작성 시간
text2 = []      # 게시물 내용
com_cnt2 = []   # 댓글 수
com_user2 = []  # 작성자 본인을 제외한 댓글 유저
com_text2 = []  # 댓글 user_id & text

In [6]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
# options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

In [7]:
# 1. 권리당원 게시판 접속
query_url= ('https://theminjoo.kr/membership/login')

driver.get(query_url)
driver.implicitly_wait(5)

In [8]:
# login

minju_id = 'gorviano'
minju_pw = 'soop*dive20'

# id 입력
driver.find_element_by_id('login_id').click()
for i in minju_id :
    time.sleep(0.1)
    driver.find_element_by_id('login_id').send_keys(i)

# pw 입력
driver.find_element_by_id('login_pw').click()
for i in minju_pw :
    time.sleep(0.1)
    driver.find_element_by_id('login_pw').send_keys(i)
    
driver.find_element_by_id('login_pw').send_keys(Keys.ENTER)
time.sleep(2)

In [9]:
# 권리당원 게시판 이동
query_url= ('https://theminjoo.kr/board/lists/rmboard?page=1')

driver.get(query_url)
time.sleep(2)

In [10]:
domain1 = "권리당원게시판"

In [11]:
# 1차 크롤링 

page_num = 1
while_code = 0

while True :
    
    if while_code == 1 :
        break
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    item_box = soup.find('ul', id='boardPostList').find_all('li', class_=False)
    
    for item in item_box :
        date1 = item.find('td', class_='date').text.replace("작성일:","")
        
        
        if "초" in date1 or "분" in date1 or "시간" in date1 :
            date1 = a
        elif "1일전" in date1 :
            tmp_date = datetime.now()-timedelta(days=to_day+1)
            tmp_date = tmp_date.strftime("%Y-%m-%d")
            date1 = tmp_date
        elif "2일전" in date1 :
            tmp_date = datetime.now()-timedelta(days=to_day+2)
            tmp_date = tmp_date.strftime("%Y-%m-%d")
            date1 = tmp_date
        else :
            date1 = "20" + date1
            
        print(date1)
        
        if date1 > a :
            pass
        
        elif date1 >= b : 

            title1 = item.find('div', class_='subject clearfix').find('a', href=True).text
            
            name1 = item.find('a', href=True).text
            url1 = "https://theminjoo.kr" + item.find('div', class_='subject clearfix').find('a', href=True)['href']
            
            print(title1)
            print(name1)
            print(url1)
            
            domain2.append(domain1)
            title2.append(title1)
            date2.append(date1)
            user2.append(name1)
            url2.append(url1)
            
        
        else :
            while_code = 1
            break
            
    page_num += 1
    driver.get(f"https://theminjoo.kr/board/lists/rmboard?page={page_num}")
    time.sleep(random.randint(1,2))

2021-05-26
도대체 니들 뭐하냐?
v일당백
https://theminjoo.kr/board/view/rmboard/509199?page=1
2021-05-26
일베스런 아들하고 집에서 코인하며 쉬세요.
쑥덕쑥덕
https://theminjoo.kr/board/view/rmboard/509198?page=1
2021-05-26
무능한 당대표 뽑은 사람들 반성하라.
향수
https://theminjoo.kr/board/view/rmboard/509197?page=1
2021-05-26
송영길대표님 3수해서 대표 됐는데
막국수
https://theminjoo.kr/board/view/rmboard/509196?page=1
2021-05-26
이재명이 숫자 10을 좋아하는 또 하나의 이유...
소공화국
https://theminjoo.kr/board/view/rmboard/509194?page=1
2021-05-26
송영길 사퇴하라
은수기다as
https://theminjoo.kr/board/view/rmboard/509193?page=1
2021-05-26
송대표 결단하라!
선한사람
https://theminjoo.kr/board/view/rmboard/509192?page=1
2021-05-26
부자들의 이익을 대변하는 당이 되었습니까?
오일러
https://theminjoo.kr/board/view/rmboard/509191?page=1
2021-05-26
할말없다
릴리리
https://theminjoo.kr/board/view/rmboard/509188?page=1
2021-05-26
국민 84% "LH 투기와 민주당의 대처가 보궐에 가장 큰 영향"
구름여름
https://theminjoo.kr/board/view/rmboard/509187?page=1
2021-05-26
코끼리는생각마!!!프레임전쟁.
메롱메롱
https://theminjoo.kr/board/view/rmboard/509186?page=1
2021-05-26
송영길씨 제발
산중일기
ht

2021-05-26
당대표는 하고싶은 일 하는 자리가 아니다
보루
https://theminjoo.kr/board/view/rmboard/509081?page=6
2021-05-26
가장 중요한 시기에 가장 악질의 당대표, 송영길은 사퇴하라 !!
까떼
https://theminjoo.kr/board/view/rmboard/509080?page=6
2021-05-26
송영길사퇴하라
lovelyanna
https://theminjoo.kr/board/view/rmboard/509079?page=6
2021-05-26
송영길은 사퇴하고 이재명은 탈당해라
osoring
https://theminjoo.kr/board/view/rmboard/509075?page=6
2021-05-26
송영길은 창당해서 신당 당대표 하세요
아기늑대
https://theminjoo.kr/board/view/rmboard/509074?page=6
2021-05-26
송영길은 사퇴하라
이니포에버
https://theminjoo.kr/board/view/rmboard/509073?page=6
2021-05-26
송영길은 사퇴하라
화성유후
https://theminjoo.kr/board/view/rmboard/509072?page=6
2021-05-26
송영길은 사퇴해라
지아코
https://theminjoo.kr/board/view/rmboard/509071?page=6
2021-05-26
송영길은사퇴하라!
어용할매
https://theminjoo.kr/board/view/rmboard/509070?page=6
2021-05-26
송영길은 사퇴하라
아즈라엘12
https://theminjoo.kr/board/view/rmboard/509069?page=6
2021-05-26
송영길은 사퇴하세요
물고기소리
https://theminjoo.kr/board/view/rmboard/509068?page=6
2021-05-26
송영길 사퇴해라
vain40
https://theminjoo.kr/board

2021-05-26
송영길 사퇴해
웨이
https://theminjoo.kr/board/view/rmboard/508963?page=11
2021-05-26
언론개혁, 포탈개혁 좀 하십시오
Laplacian
https://theminjoo.kr/board/view/rmboard/508962?page=11
2021-05-26
많은 비판들이 있지만 민주당 항상 지지합니다
이자벨송
https://theminjoo.kr/board/view/rmboard/508961?page=11
2021-05-26
당대표 자격없는 송영길은 사퇴하고 이재명,이해찬은 탈당하라!!
khs
https://theminjoo.kr/board/view/rmboard/508960?page=11
2021-05-26
민주야 다시 잘 하자 그 전에 이낙연과 초선 5인은 좀 맞고 하자 + 고영인 + 김진표
피의화요일
https://theminjoo.kr/board/view/rmboard/508959?page=11
2021-05-26
노선 잘못 잡으셨습니다
로시
https://theminjoo.kr/board/view/rmboard/508958?page=11
2021-05-26
송영길 당대표 사퇴하세요!!!
프테라노돈
https://theminjoo.kr/board/view/rmboard/508957?page=11
2021-05-26
송영길당대표님 꾸김당 엑스맨 입니까
낙지gout19
https://theminjoo.kr/board/view/rmboard/508956?page=11
2021-05-26
노무현 후윈재단 의 그림
진보적폐없애자
https://theminjoo.kr/board/view/rmboard/508955?page=11
2021-05-26
송영길 사퇴하세요
더불어민주당원입니당
https://theminjoo.kr/board/view/rmboard/508954?page=11
2021-05-26
송영길 이재명 이해찬 탈당하라
요술램프
https://theminjoo.kr/board/view/r

2021-05-26
송영길은 어느나라 당대표냐? 상황봐서 대통령보고 탈당하라 그럴기세네
네루
https://theminjoo.kr/board/view/rmboard/508846?page=16
2021-05-26
당대표때문에 묻힌사람 누구
never93
https://theminjoo.kr/board/view/rmboard/508845?page=16
2021-05-26
한미정상회담 백업도 안하는 송영길은 사퇴해라
tkfkadlajswj
https://theminjoo.kr/board/view/rmboard/508844?page=16
2021-05-26
송영길 대표는 어느당 당대표인가요?
aann
https://theminjoo.kr/board/view/rmboard/508843?page=16
2021-05-26
송영길 사퇴하세요
강남구
https://theminjoo.kr/board/view/rmboard/508842?page=16
2021-05-26
송영길 사퇴해!
호랑이13
https://theminjoo.kr/board/view/rmboard/508841?page=16
2021-05-26
잃어버린 2년(by 이해찬)
낙지사랑털보
https://theminjoo.kr/board/view/rmboard/508840?page=16
2021-05-26
송영길 사퇴하라!
민주dem12
https://theminjoo.kr/board/view/rmboard/508839?page=16
2021-05-26
송영길은 야당 대표입니까?
깨어나는봄
https://theminjoo.kr/board/view/rmboard/508838?page=16
2021-05-26
반문질하는 것들은 다 꺼지세요!!!!
섹시마녀
https://theminjoo.kr/board/view/rmboard/508836?page=16
2021-05-26
송영길이 민주당 대표 맞나요?
미스터선샤인
https://theminjoo.kr/board/view/rmboard/508835?page=16
2021-0

2021-05-26
0.59로 간신히 된 송영길, 문프 공격하려고 당대표 됐구만
보라루비
https://theminjoo.kr/board/view/rmboard/508725?page=21
2021-05-26
쩜오 당댚이라 그모양인가?
문is진리
https://theminjoo.kr/board/view/rmboard/508724?page=21
2021-05-26
민주당은 미끄럼 탔어요!
민주당재조산하
https://theminjoo.kr/board/view/rmboard/508723?page=21
2021-05-26
진짜 너무 답답하네요
연지곤지
https://theminjoo.kr/board/view/rmboard/508722?page=21
2021-05-26
송영길 뽑으신 분들 밥은 잘 먹고 있는지
봉하마을이장
https://theminjoo.kr/board/view/rmboard/508721?page=21
2021-05-26
오랜만에 국뽕 모드
MOS6502
https://theminjoo.kr/board/view/rmboard/508720?page=21
2021-05-26
송영길
예쁜콩
https://theminjoo.kr/board/view/rmboard/508719?page=21
2021-05-26
이재명 이해찬 송영길 나가!!!!!!!!!!!!!!!!!!! 나가!!!!!!!!!!!!!!!!!!!
어이상실
https://theminjoo.kr/board/view/rmboard/508716?page=21
2021-05-26
송영길 대표 여당대표답게 행동하세요!
김기철
https://theminjoo.kr/board/view/rmboard/508715?page=21
2021-05-26
그렇게 114라고 정답을 알려 줬는데
Nomad
https://theminjoo.kr/board/view/rmboard/508714?page=21
2021-05-26
송영길 당대표님!  제발  정신차리세요!
이헌세
https://theminjoo.kr/board/view/r

...

2021-05-24
송대표 보면 조순형이 떠오르네요
푸루룽
https://theminjoo.kr/board/view/rmboard/508245?page=41
2021-05-24
혹시 송영길씨가 자한당에 또있나요??^^;;
더치아메리카노
https://theminjoo.kr/board/view/rmboard/508244?page=41
2021-05-24
송영길 대표님. 제발  ....
샘물처럼
https://theminjoo.kr/board/view/rmboard/508243?page=41
2021-05-24
진짜 입조심좀 하세요 당대표란 인간이 입이 아니라 주둥아리를 쓰는게 말이 됩니까?
인간형코코몽34호
https://theminjoo.kr/board/view/rmboard/508242?page=41
2021-05-24
송영길 진짜 쓰*기 다...
웰컴
https://theminjoo.kr/board/view/rmboard/508240?page=41
2021-05-24
송영길 대표님... 내년 대선 말아 먹으려고 그러나요?
민주당잘합시다
https://theminjoo.kr/board/view/rmboard/508239?page=41
2021-05-24
송영길씨는 민주당 사람 맞나요?
퍼니어
https://theminjoo.kr/board/view/rmboard/508238?page=41
2021-05-24
일만 잘 하면 된다며 경기도 도정평가 들이미는 분들
PDS
https://theminjoo.kr/board/view/rmboard/508237?page=41
2021-05-24
당대표님 보십쇼
남극12
https://theminjoo.kr/board/view/rmboard/508236?page=41
2021-05-24
게시판에 처음 글쓰네요^^
국민중한사람
https://theminjoo.kr/board/view/rmboard/508235?page=41
2021-05-24
주제를 모르는구나 진짜
공돌이
https://theminjoo.kr/board/vi

2021-05-24
#이제명 나가라!!!
사사세71
https://theminjoo.kr/board/view/rmboard/508122?page=46
2021-05-24
이낙연지지자분들은 게시판 도배를 멈추어 주십시오.
젬스정
https://theminjoo.kr/board/view/rmboard/508121?page=46
2021-05-24
민주당 망하려고 아주 작정을 했다
오헤일리
https://theminjoo.kr/board/view/rmboard/508120?page=46
2021-05-24
당대표님 야당대표세요?
모찌누나
https://theminjoo.kr/board/view/rmboard/508119?page=46
2021-05-24
송영길 이상하네
달달달달
https://theminjoo.kr/board/view/rmboard/508118?page=46
2021-05-24
인천 21세기병원 문제.....
action4321
https://theminjoo.kr/board/view/rmboard/508115?page=46
2021-05-24
이재명아웃) 당대표는 왜 있는 자리인가 모르겠네요.
쓰레빠83
https://theminjoo.kr/board/view/rmboard/508113?page=46
2021-05-24
송영길 대표님 당을 망치려고 작정했나요?
날개달자
https://theminjoo.kr/board/view/rmboard/508112?page=46
2021-05-24
이준석에게 짝궁 만들어줍시다. 김한규, 최지은 조합 어떤가요?
옵티션
https://theminjoo.kr/board/view/rmboard/508110?page=46
2021-05-24
사람사는 사회에서 장유유서란
군포장
https://theminjoo.kr/board/view/rmboard/508109?page=46
2021-05-24
이낙연 의원님 보고싶어요
화조도
https://theminjoo.kr/board/view/rmboard/508108?page=46

2021-05-25
유시민    노무현재단에서 나가주세요
Tree63
https://theminjoo.kr/board/view/rmboard/507995?page=51
2021-05-25
사이다 좋아하면  이빨 다 썩어요
딸랑이
https://theminjoo.kr/board/view/rmboard/507994?page=51
2021-05-25
뭣이 중한지...
전라불패
https://theminjoo.kr/board/view/rmboard/507993?page=51
2021-05-25
송영길은 왜 국짐2중대를 자처하는가
깨끗한
https://theminjoo.kr/board/view/rmboard/507992?page=51
2021-05-25
0.59퍼센트차 당대표 수준
WhiteSue
https://theminjoo.kr/board/view/rmboard/507989?page=51
2021-05-25
코인과 부동산에 꽂혀 투기 부추기는 당대표
안티고네
https://theminjoo.kr/board/view/rmboard/507988?page=51
2021-05-25
송영길이 사는 길
백년불패
https://theminjoo.kr/board/view/rmboard/507987?page=51
2021-05-25
이래서 송영길 당대표 아니였다!!!
cowtree1
https://theminjoo.kr/board/view/rmboard/507986?page=51
2021-05-25
이낙연의원님 지지합니다!!
황토오소리
https://theminjoo.kr/board/view/rmboard/507985?page=51
2021-05-25
송영길은 왜
월인천강
https://theminjoo.kr/board/view/rmboard/507984?page=51
2021-05-25
대표라는 분이 왜 그리 입을 가볍게 놀리시나..
septblue
https://theminjoo.kr/board/view/rmboard/507979?page=51
2021-05-25
송영길 대표

2021-05-24
정치 원칙
민주당똑바로일해자
https://theminjoo.kr/board/view/rmboard/507856?page=56
2021-05-24
미래 세대에게 희망이 있는 나라가 돼야 ㅡㅡㅡ
pjs1
https://theminjoo.kr/board/view/rmboard/507855?page=56
2021-05-24
답은 나왔다
수철사오
https://theminjoo.kr/board/view/rmboard/507853?page=56
2021-05-24
이재명 지지자들이 경기포럼에 발작일으키는 이유
바라모스
https://theminjoo.kr/board/view/rmboard/507852?page=56
2021-05-24
쌍욕 녹음 파일) 대부분 사람들 용납 못함!
아녜스이모
https://theminjoo.kr/board/view/rmboard/507851?page=56
2021-05-24
이낙연 전대표님 이재명 안방 경기서 2만명 초대형 포럼 출범
나오소리
https://theminjoo.kr/board/view/rmboard/507850?page=56
2021-05-24
전투형 노무현이 아니라 조폭형 이명박이 맞지않나?
쭈니천사
https://theminjoo.kr/board/view/rmboard/507849?page=56
2021-05-24
대체 이석기랑은 무슨 사이이신지요
PDS
https://theminjoo.kr/board/view/rmboard/507848?page=56
2021-05-24
박주민의원이 법사위원장으로 적격임.
진실로
https://theminjoo.kr/board/view/rmboard/507846?page=56
2021-05-24
아래 “리쉬”가 저의 글을 보고 제목을 바꿨네요 당원님들의 이해를 돕기 위해 사진을 올려드립니다
Ilovepurin
https://theminjoo.kr/board/view/rmboard/507845?page=56
2021-05-24
그지사네 왠지 실제는 한줌인거 같지 않냐
김

2021-05-24
쌍욕 감당 못합니다.
섀도우
https://theminjoo.kr/board/view/rmboard/507714?page=61
2021-05-24
이낙연 정말 인간 말종이었네 18
불광동휘발류
https://theminjoo.kr/board/view/rmboard/507713?page=61
2021-05-24
김진표씨는 부동산특위장울 내놓아야 합니다.
tlsdlstjq77
https://theminjoo.kr/board/view/rmboard/507712?page=61
2021-05-24
문정부와 당원의 길
김스티브
https://theminjoo.kr/board/view/rmboard/507710?page=61
2021-05-24
대통령님께 빌붙어사는 불어터진 민주당아!!!
시유81
https://theminjoo.kr/board/view/rmboard/507707?page=61
2021-05-24
민주당을 지지하는 이유는 이재명이 있어서가 아니다.
고양이손
https://theminjoo.kr/board/view/rmboard/507704?page=61
2021-05-24
바이든이 음주운전 논문표절 패륜잡범을 어캐 대할까?
평타
https://theminjoo.kr/board/view/rmboard/507703?page=61
2021-05-24
선진 대한민국 품격있는 문재인대통령님 자랑스럽습니다.
stellarQ
https://theminjoo.kr/board/view/rmboard/507701?page=61
2021-05-24
판문점 선언 비준하세요.
오일러
https://theminjoo.kr/board/view/rmboard/507700?page=61
2021-05-24
탈당 합니다
칼키
https://theminjoo.kr/board/view/rmboard/507699?page=61
2021-05-24
민주당 의원님들!  언론 개혁 절대 늦추면 안 됩니다. (한미정상회담 기사를 포털에서 볼 수가 없습니다.민주당의 반응은 찾아보려해

In [12]:
def scroll_down() :
    element = driver.find_element_by_tag_name('body')
    element.send_keys(Keys.END)
    element.send_keys(Keys.PAGE_DOWN)

In [ ]:
# 2차 크롤링

for url in tqdm(url2) :
    driver.get(url)
    scroll_down()
    time.sleep(1)

    while True :

        try :
            scroll_down()
            driver.find_element_by_class_name("more_btn").click() 
        except :
            break
        
        time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    text1 = soup.find('div', class_='bv_middle').text.strip()
    print(text1)

    com_box = soup.find('ul', id='boardCmtLists').find_all('li')

    com_user1 = []
    com_text1 = []
    for com in com_box :
        com_name = com.find('div', class_='mem_nick').text
        com_text = com.find('div', class_='reply_cnt').text

        if "님에 의해 삭제된 글입니다." in com_text :
            pass
        else :
            com_user1.append(com_name)
            com_text1.append(f"작성자 : {com_name} :: {com_text}")
            
    print(com_user1)
    print(com_text1)
    text2.append(text1)
    com_user2.append(com_user1)
    com_text2.append(com_text1)

  0%|                                                                               | 1/1281 [00:07<2:35:21,  7.28s/it]

﻿진짜 머저리같은 짓만 할래? 진짜 기득권정당이 되가는구나 김진표같은 넘을 부동산특위 위원장으로 하니 정책이 그따위지 진짜 욕나오게 하네 도대체 부자들 세금깎아주는 정책을 내면서 민주당 지지해달라 할수있니? 송영길 대표 니정치 하지말고 대선관리나 제대로해라 민주당 말아먹을려고 작정한 넘도 아니고
[]
[]


  0%|                                                                               | 2/1281 [00:14<2:34:58,  7.27s/it]

﻿그리고 다 털리면 정부에 험한 욕이나 하고
[]
[]


  0%|▏                                                                              | 3/1281 [00:21<2:34:46,  7.27s/it]

...

﻿이낙연 왈:노무현 하면 희망 떠오른다.인간 말종시키.임기내내 온갖 더러운짓거리 다 해놓고이제와서 희망이 보인단다. C벨넘﻿
['Tomato08', 'nagodory', 'go사랑go', '신길동터쟁이', '금구66', '서머', '까칠한미니', '민주네', '영철이', '비아주', '산그림자로88', '품격민주', 'Thetheminjoo']
['작성자 : Tomato08 :: \n                            반노짓하던 정통회장이 뭐?\n인간말종은 느그 지사님\n\n\n이재명에게서 정동영이 보인다\n ', '작성자 : nagodory :: \n                            진짜 가짜뉴스 짜집기 왜곡 날조는 느그 도지사가 갈쳐주더나? 어디서 저급하게 쌍욕이야 무식하고 천박하기가                        ', '작성자 : go사랑go :: \n                            정통 회장이 누구였더라. 혜경궁 트윗은 괜찮으시죠?                        ', '작성자 : 신길동터쟁이 :: \n                            이야 말 *같이 하네                        ', '작성자 : 금구66 :: \n                            너는 포털에 출근해서 사사건건 반문질하대...역시 이재명 졸개답다                        ', '작성자 : 서머 :: \n                            저급한 단어좀 삼가해주시구요\n아무리 그래도 가족에게 쌍욕하는 누구보단 백배 나은분이니 상식적으로 생각하시길요                        ', '작성자 : 까칠한미니 :: \n                            어디서 가짜 뉴스를...\n쓰레기 가짜 휘발유 아닌가 ㅉㅉ                        ', '작성자 : 민주네 :: \n                    

  7%|█████▌                                                                        | 91/1281 [11:03<2:26:01,  7.36s/it]

더불어 이재명은 탈당하라!!!
[]
[]


  7%|█████▌                                                                        | 92/1281 [11:10<2:25:26,  7.34s/it]

﻿진짜 우려를 했지만이리도 빨리 뒤통수를 치다니쩜오로 대표된지 잉크도 안말랐겠다.기다렸다는 듯이 정부 들이 받는대표를 보게 될 줄이야.고쳐쓰는 것 아니라고 했지만 그래도 일말의 기대를 했건만여당대표 입에서는 나와서는 안되는 말을서슴없이 하는 당대표라니빨리 내려와라더 이상 당 망치지말고
[]
[]


In [ ]:
df = pd.DataFrame()

df['domain'] = pd.Series(domain2)
# df['keyword'] = pd.Series(keyword2)
df['title'] = pd.Series(title2)
df['user_name'] = pd.Series(user2)
df['url'] = pd.Series(url2)
df['date'] = pd.Series(time2)
df['view_cnt'] = pd.Series(view_cnt2)
df['like_cnt'] = pd.Series(like_cnt2)
df['user_text'] = pd.Series(text2)
df['com_cnt'] = pd.Series(com_cnt2)
df['com_user'] = pd.Series(com_user2)
df['com_text'] = pd.Series(com_text2)

df

In [ ]:
# csv 형태로 저장하기
df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")